In [ ]:
import pyaudio
import numpy as np
import winsound
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText

# Function to calculate dB level from audio data
def calculate_decibel_level(audio_data):
    # Calculate RMS (Root Mean Square) of audio data
    rms = np.sqrt(np.mean(np.square(audio_data)))
    # Reference level for dB calculation
    ref_level = 1.0
    # Calculate dB level
    decibel_level = 20 * np.log10(rms / ref_level)
    return decibel_level

# Function to check for abnormal sound level
def check_sound_level(stream):
    CHUNK = 1024  # Number of frames per buffer

    while True:
        # Read audio data from the stream
        data = stream.read(CHUNK)
        # Convert data to numpy array
        audio_data = np.frombuffer(data, dtype=np.int16)
        # Calculate dB level
        decibel_level = calculate_decibel_level(audio_data)
        print("Decibel level:", decibel_level)

        # If decibel level exceeds a certain threshold, trigger alert
        if decibel_level > 35:  # Adjust threshold as needed
            print("Abnormal sound detected! Triggering alert...")
            # Trigger beep sound
            winsound.Beep(1000, 3000)  # Frequency: 1000 Hz, Duration: 1000 ms
            # Send email alert
            flag = send_email_alert()

# Function to send email alert
def send_email_alert():
    # Email configuration
    fromaddr = '2021ad0610@svce.ac.in'
    toaddrs  = 's.gokul13082003@gmail.com'
    message = MIMEMultipart()
    message["From"] = fromaddr
    message["To"] = toaddrs
    message["Subject"] ="URGENT: Emergency Attention Required for Patients"
    body = '''🚨🚨🚨ALERT!!!🚨🚨🚨

Emergency attention needed on the patients, currently under our care.

Thank you,
Regards
Soundscape Monitor'''
    message.attach(MIMEText(body, "plain"))

    username = '2021ad0610@svce.ac.in'
    password = 'Deepanramesh2003$'

    server = smtplib.SMTP('smtp.gmail.com', 587)
    server.ehlo()
    server.starttls()
    server.login(username, password)
    server.sendmail(fromaddr, toaddrs, message.as_string())
    print("Alert mail sent")
    server.quit()
    return True


# Main function
def main():
    p = pyaudio.PyAudio()
    # Open audio stream
    stream = p.open(format=pyaudio.paInt16,
                    channels=1,
                    rate=44100,
                    input=True,
                    frames_per_buffer=1024)
    try:
        # Call function to check sound level
        a = check_sound_level(stream)
        if a==True:
            stream.stop_stream()
            stream.close()
            p.terminate()
    except KeyboardInterrupt:
        # Close the audio stream when Ctrl+C is pressed
        stream.stop_stream()
        stream.close()
        p.terminate()

if __name__ == "__main__":
    main()
